### Load synonym data and current rule data
- ps: 分析用的規則是 `rule_dataset_final.csv`，包含完整的 system calls。實作上沒有用。
- `synonym_dataset.csv`

In [27]:
import pandas as pd

rules = pd.read_csv('../rule_dataset_final.csv')

def clean_braces(x):
    x = x.replace("()", "").replace(",", "").replace(" ", "")
    return str(x)
def add_braces(x):
    x = clean_braces(x)
    x = x+"()"
    return x

rules['Syscall'] = rules['Syscall'].apply(add_braces) #clean_braces
rules.head(3)

,EntityType,ActionType,Source,Syscall,EnVerb,Sentence
0,DEVICE,DEVICE,man,io_cancel(),cancel,cancel an outstanding asynchronous I/O operation
1,DEVICE,DEVICE,man,io_destroy(),destroy,destroy an asynchronous I/O context
2,DEVICE,DEVICE,man,io_getevents(),read,read asynchronous I/O events from the completi...


In [5]:
synbase = pd.read_csv('../synonym_data/synonym_dataset.csv')
synbase = synbase[~synbase['Syscall'].isin(['rm()', 'exec()'])]
display(synbase.head(3))

# 只有一個 Synonym 的 syscall: 115 個
def count_pairs(x):
    numOfSplit = len(x.split('|'))
    if numOfSplit == 1 and (x == '' or x == '<,>'):
        return 0
    return numOfSplit

# dataset['numOfSynonym'] = dataset['Synonym'].apply(count_pairs)
# dataset['numOfFirstLine'] = dataset['FirstLine'].apply(count_pairs)

def extract_verb(x:str):
    ''' <,> => None, <get,xxx> => get '''
    x = x.replace('<','').replace('>','')
    x = x.split(',')[0]
    return x.strip()

# dataset['FirstVerb'] = dataset['FirstLine'].apply(extract_verb)

,Syscall,Sentence,FirstLine,Synonym,SynonymSentence
0,shutdown(),shut down part of a full-duplex connection,"<shut down,part of a full-duplex connection>","<,>",NaN
1,recvmmsg(),receive multiple messages on a socket,"<receive,multiple messages>","<,>",NaN
2,sendmmsg(),send multiple messages on a socket,"<send,multiple messages>","<transmit,multiple messages>",transmit multiple messages on a socket using a...


### Extract verbs and sentence from synbase

In [19]:
syndata = {}
for i, synrow in synbase.iterrows():
    verbs = set()
    # print(synrow)
    # extract_verb(synrow['FirstLine'])
    verbs.add(extract_verb(synrow['FirstLine'])) # name sector
    if synrow['Synonym'] != '<,>':
        des = synrow['Synonym'].split('|') # description sector
        [verbs.add(extract_verb(d)) for d in des]
    syscall = synrow['Syscall']
    if len(verbs) == 0:
        continue
    try:
        syndata[syscall].update(verbs)
    except:
        syndata[syscall] = set(verbs)

from itertools import islice
def take(n, iterable):
    """Return the first n items of the iterable as a list."""
    return list(islice(iterable, n))

n_items = take(5, syndata.items())
n_items

[('shutdown()', {'shut down'}),
 ('recvmmsg()', {'receive'}),
 ('sendmmsg()', {'send', 'transmit'}),
 ('pipe2()', {'create'}),
 ('mq_unlink()', {'remove'})]

依照 synbase 建立新的 rule dataset

In [35]:
def getType(syscall:str, ruledf:pd.DataFrame):
    ''' search in rule dataset and get 'EntityType', 'ActionType'.'''
    entity, action = None, None
    rules = ruledf[ruledf['Syscall'] == syscall]
    if len(rules) != 0:
        rules = rules.reset_index()
        # display(rules)
        entity = rules.loc[0, 'EntityType']
        action = rules.loc[0, 'ActionType']
    return entity, action

getType('sendmmsg()', rules)

('NET', 'NET_TRANSMIT')

In [36]:
ruledata = []
for syscall, verbset in syndata.items():
    entity, action = getType(syscall, rules)
    for verb in verbset:
        ruledata.append({
            'EntityType': entity,
            'ActionType': action,
            'Source': 'man', 
            'Syscall': syscall, 
            'EnVerb': verb, 
            'Sentence': None,
        })
df = pd.DataFrame(ruledata) # df is the new rule dataset for statistic
print('num of syscall', len(df['Syscall'].unique()),', num of verb',  len(df['EnVerb'].unique()))
df

num of syscall 395 , num of verb 136


,EntityType,ActionType,Source,Syscall,EnVerb,Sentence
0,NET,NET_CLOSE,man,shutdown(),shut down,None
1,NET,NET_TRANSMIT,man,recvmmsg(),receive,None
2,NET,NET_TRANSMIT,man,sendmmsg(),transmit,None
3,NET,NET_TRANSMIT,man,sendmmsg(),send,None
4,NET,NET_CONNECT,man,pipe2(),create,None
...,...,...,...,...,...,...
563,NET,NET_TRANSMIT,man,recvfrom(),receive,None
564,NET,NET_TRANSMIT,man,send(),send,None
565,NET,NET_TRANSMIT,man,sendto(),send,None
566,NET,NET_TRANSMIT,man,sendmsg(),send,None


In [40]:
# 輸出 csv 並標註
# df.to_csv('need_label.csv', index=False) 
# 重新 sort 規則
df = pd.read_csv('need_label.csv')
df = df.sort_values(by=['EntityType', 'ActionType', 'Syscall'])
df.to_csv('after_label.csv', index=False) 

### add verb from CTI report back to rule dataet

- 現在新的 rule_dataset 已經建好了(從 synbase 把手冊翻譯轉過來)，把從 CTI 找到的動詞加過去

In [45]:
df =pd.read_csv('after_label.csv') 
rules_from_cti = rules[rules['Source'] != 'man']
df = pd.concat([df, rules_from_cti])
df = df.sort_values(by=['EntityType', 'ActionType', 'Syscall'])

print('num of syscall', len(df['Syscall'].unique()),', num of verb',  len(df['EnVerb'].unique()))
df.head()

num of syscall 396 , num of verb 155


,EntityType,ActionType,Source,Syscall,EnVerb,Sentence
0,DEVICE,DEVICE,man,io_cancel(),cancel,NaN
1,DEVICE,DEVICE,man,io_destroy(),destroy,NaN
2,DEVICE,DEVICE,man,io_getevents(),read,NaN
3,DEVICE,DEVICE,man,io_setup(),create,NaN
4,DEVICE,DEVICE,man,io_submit(),submit,NaN


In [46]:
df.to_csv('rule_dataset_final.csv', index=False)

- Next step，需要人工刪除 from 手冊和 from CTI 重複的動詞
- 例如說 nanosleep() 的 sleep 動詞在手冊跟 CTI 都有重複 => 刪除 CTI 的這行